### 🚀 Day 2-3: 첫 분류 모델 구축하기 - 로지스틱 회귀 & K-NN

지난 시간에는 머신러닝 모델의 성능을 좌우하는 가장 중요한 단계 중 하나인 **데이터 전처리**에 대해 배웠습니다. 

결측치를 채우고, 범주형 변수를 숫자로 변환하고, 데이터의 스케일을 맞추는 등 '정제된 재료'를 준비하는 과정을 마쳤습니다.

이제 우리는 잘 준비된 데이터를 가지고, 고객이 이탈할지 아닐지를 예측하는 첫 **분류(Classification)** 모델을 직접 만들어 볼 차례입니다. 

이번 시간에는 가장 기본적이면서도 강력한 두 가지 분류 알고리즘, **로지스틱 회귀(Logistic Regression)** 와 **K-최근접 이웃(K-Nearest Neighbors, K-NN)** 에 대해 배우게 됩니다.

두 모델은 문제를 해결하는 접근 방식이 완전히 다릅니다.

* **로지스틱 회귀** 는 데이터가 특정 클래스에 속할 **확률** 을 계산하여 예측하는, 통계에 기반한 선형 모델입니다.
  
* **K-NN** 은 "끼리끼리 모인다"는 단순한 아이디어에서 출발하여, 새로운 데이터 주변의 **가장 가까운 'k'개 이웃** 을 보고 클래스를 결정하는, 거리 기반 모델입니다.

이번 챕터를 통해 우리는 각 모델의 작동 원리를 이해하고, `scikit-learn`으로 직접 구현해 볼 것입니다. 

또한, 모델의 성능을 결정하는 **하이퍼파라미터** 를 조정하고, 모델이 어떻게 클래스를 구분하는지 보여주는 **결정 경계(Decision Boundary)** 를 시각화하여 모델의 '생각'을 엿보는 흥미로운 경험을 하게 될 것입니다.

이번 실습에서도 **통신사 고객 이탈(Telco Customer Churn) 데이터셋** 을 계속 사용하며, Part 2에서 구축한 전처리 파이프라인을 적극적으로 활용할 것입니다. 그럼, 첫 분류 모델을 향한 여정을 시작해볼까요?


---

### 1. 실습 환경 및 데이터 준비

본격적인 모델링에 앞서, 이전 파트에서 완성한 전처리 파이프라인을 포함한 기본 코드를 준비합니다.

이 파이프라인은 수치형 데이터와 범주형 데이터를 각각 다르게 처리하여 모델이 학습할 수 있는 형태로 만들어줍니다.

#### 💻 코드

In [1]:
# --- 기본 라이브러리 임포트 ---
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [2]:

# --- scikit-learn 도구 임포트 ---
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.decomposition import PCA

In [3]:
# --- 데이터셋 로드 및 기본 전처리 ---
path = '../datasets/ml/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv' # 파일 경로는 환경에 맞게 조정하세요.
df = pd.read_csv(path)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.drop('customerID', axis=1, inplace=True)
df[['TotalCharges']].head()


,TotalCharges
0,29.85
1,1889.50
2,108.15
3,1840.75
4,151.65


In [6]:
# --- 특성과 타겟 분리 ---
X = df.drop('Churn', axis=1)
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0) # 타겟을 0과 1로 변환

# --- 훈련/테스트 데이터 분리 ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
# --- 전처리 파이프라인 (Day 2-2에서 구축) ---
# 수치형 특성 식별
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
# 범주형 특성 식별
categorical_features = X.select_dtypes(include='object').columns.tolist()

# 수치형 변수 전처리 파이프라인
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 변수 전처리 파이프라인
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer를 사용하여 두 파이프라인 통합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

---

### 2. 로지스틱 회귀 (Logistic Regression)

이름에 '회귀(Regression)'가 들어가 있지만, 로지스틱 회귀는 분류 문제를 해결하는 데 널리 사용되는 핵심 알고리즘입니다. 

데이터가 특정 클래스에 속할 확률을 예측하고, 그 확률을 바탕으로 클래스를 결정합니다.

#### 🧠 개념 이해하기

로지스틱 회귀의 핵심 아이디어는 **시그모이드 함수(Sigmoid Function)** 에 있습니다. 

선형 회귀가 예측값을 직선 형태로 출력하는 반면, 로지스틱 회귀는 선형 함수의 결과를 시그모이드 함수에 통과시켜 0과 1 사이의 값, 즉 **확률** 로 변환합니다.

$$S(z) = \frac{1}{1 + e^{-z}}$$

<img src="https://blog.kakaocdn.net/dn/sib2q/btsDJkfaBMy/pkKQYrLYXHVDODrXvQNqK0/img.png">

여기서 $z$는 입력 특성(feature)들의 선형 조합($z = w_0 + w_1x_1 + ... + w_nx_n$)입니다.

* $z$가 매우 큰 양수이면 $e^{-z}$는 0에 가까워지고, $S(z)$는 1에 가까워집니다.
* $z$가 매우 작은 음수이면 $e^{-z}$는 무한대에 가까워지고, $S(z)$는 0에 가까워집니다.
* $z$가 0이면 $S(z)$는 정확히 0.5가 됩니다.

이렇게 계산된 확률값을 바탕으로, 우리는 **결정 경계(Decision Boundary)** 라는 기준(보통 0.5)을 사용하여 클래스를 최종적으로 결정합니다. 

예를 들어, 시그모이드 함수 출력이 0.5 이상이면 '이탈(1)', 0.5 미만이면 '비이탈(0)'로 예측하는 식입니다.

로지스틱 회귀는 각 특성에 대한 **계수(coefficient)** 를 학습하는데, 이 계수 값은 해당 특성이 결과에 얼마나 큰 영향을 미치는지(중요도)와 긍정적인 영향인지 부정적인 영향인지를 알려주는 중요한 지표가 됩니다.

#### 💻 코드로 알아보기

이제 `scikit-learn`의 `Pipeline`을 사용하여 전처리 과정과 로지스틱 회귀 모델을 하나로 묶어 학습시켜 보겠습니다.

In [33]:
# 로지스틱 회귀 모델을 포함하는 최종 파이프라인 구축
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))
])

# 파이프라인 학습
lr_pipeline.fit(X_train, y_train)

# 예측 및 정확도 평가
y_pred_lr = lr_pipeline.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"로지스틱 회귀 모델 정확도: {accuracy_lr:.4f}")

로지스틱 회귀 모델 정확도: 0.8055


모델의 계수를 시각화하여 어떤 특성이 고객 이탈에 큰 영향을 미치는지 확인해볼 수 있습니다.

In [19]:
# 파이프라인에서 학습된 특성 이름 가져오기
# OneHotEncoder가 적용된 범주형 특성 이름 추출
ohe_feature_names = lr_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
ohe_feature_names


array(['gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes',
       'Dependents_No', 'Dependents_Yes', 'PhoneService_No',
       'PhoneService_Yes', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service',
       'TechSupport_Yes', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month',
       'Contract_One year', 'Contract_Two year', 'PaperlessBilling_No',
       'PaperlessBilling_Yes', 'Payment

In [20]:
# 수치형 특성 이름과 합치기
all_feature_names = np.concatenate([numeric_features, ohe_feature_names])

In [21]:
# 파이프라인에서 학습된 로지스틱 회귀 모델의 계수 가져오기
coeffs = lr_pipeline.named_steps['classifier'].coef_[0]
coeffs

array([ 0.0537084 , -1.23622347, -0.55761927,  0.51181599, -0.18594614,
       -0.16400886, -0.18556076, -0.16439424, -0.06239472, -0.28756028,
       -0.14830908, -0.20164593, -0.29003419, -0.14830908,  0.08838826,
       -0.65282034,  0.60913538, -0.30627005,  0.14477885, -0.30627005,
       -0.18846381,  0.0204946 , -0.30627005, -0.06417956, -0.04733005,
       -0.30627005,  0.00364509,  0.11933782, -0.30627005, -0.16302277,
       -0.22531528, -0.30627005,  0.18163032, -0.22556157, -0.30627005,
        0.18187662,  0.56788412, -0.13165901, -0.78618012, -0.36102837,
        0.01107337, -0.19533435, -0.22487268,  0.18911597, -0.11886395])

In [22]:
# 계수를 데이터프레임으로 만들고 정렬
coeff_df = pd.DataFrame(zip(all_feature_names, coeffs), columns=['Feature', 'Coefficient'])
coeff_df['abs_coeff'] = np.abs(coeff_df['Coefficient'])
coeff_df_sorted = coeff_df.sort_values('abs_coeff', ascending=False).head(15)
coeff_df_sorted

,Feature,Coefficient,abs_coeff
1,tenure,-1.236223,1.236223
38,Contract_Two year,-0.786180,0.786180
15,InternetService_DSL,-0.652820,0.652820
16,InternetService_Fiber optic,0.609135,0.609135
36,Contract_Month-to-month,0.567884,0.567884
2,MonthlyCharges,-0.557619,0.557619
3,TotalCharges,0.511816,0.511816
39,PaperlessBilling_No,-0.361028,0.361028
22,OnlineBackup_No internet service,-0.306270,0.306270
28,TechSupport_No internet service,-0.306270,0.306270


In [23]:

# Plotly로 계수 시각화
fig = px.bar(coeff_df_sorted,
             x='Coefficient',
             y='Feature',
             orientation='h',
             title='로지스틱 회귀 모델의 주요 특성 계수',
             color='Coefficient',
             color_continuous_scale='Viridis')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

#### ✏️ 연습문제 1: 로지스틱 회귀 `solver` 변경하기

`LogisticRegression`의 `solver`는 최적의 계수를 찾는 데 사용되는 알고리즘을 지정하는 하이퍼파라미터입니다. 

`solver='liblinear'`를 `'lbfgs'`로 변경하여 파이프라인을 다시 학습시키고 정확도를 비교해보세요.


**로지스틱 회귀 Solver 비교**
 - **liblinear**: 작은 데이터셋에 적합하며, L1/L2 규제를 모두 지원합니다. 좌표 하강법을 사용합니다.
 - **lbfgs**: 중간~큰 데이터셋에 효율적이며, L2 규제만 지원합니다. 유사 뉴턴 방법을 사용하여 빠른 수렴이 가능합니다.

예시: `LogisticRegression(solver='lbfgs')`


In [ ]:
# 연습문제 1 코드

#### ✏️ 연습문제 2: 규제(Regularization) 강도 조절하기

로지스틱 회귀에서 `C`는 규제 강도를 조절하는 하이퍼파라미터입니다.

 `C` 값이 작을수록 규제가 강해져 모델이 단순해지고(과적합 방지), 클수록 규제가 약해져 훈련 데이터에 더 적합됩니다.
 
  `C` 값을 `0.1`, `1`, `10`으로 변경하며 모델을 각각 학습시키고 정확도를 비교해보세요.

  예시: `LogisticRegression(solver='liblinear', C=1)`

In [ ]:
# 연습문제 2 코드

---

### 3. K-최근접 이웃 (K-Nearest Neighbors, K-NN)

K-NN은 가장 직관적인 머신러닝 알고리즘 중 하나입니다. 

예측하려는 새로운 데이터가 주어지면, 기존 훈련 데이터 중에서 가장 가까운 'k'개의 이웃을 찾습니다. 

그리고 그 이웃들이 가장 많이 속한 클래스로 새로운 데이터의 클래스를 예측합니다.

<img src="https://blog.kakaocdn.net/dn/I6kNm/btruNidgs1i/CTokD5WZCccW7yFqa7PXtk/img.png">

#### 🧠 개념 이해하기

K-NN의 핵심 개념은 다음과 같습니다.

1.  **거리 측정(Distance Metric)**: 두 데이터 포인트가 얼마나 '가까운지'를 측정하는 방법입니다.
   
    가장 흔하게는 **유클리드 거리(Euclidean Distance)** 가 사용됩니다.

    $$d(p, q) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + ... + (q_n - p_n)^2}$$

2.  **이웃의 수 'k'**: 몇 개의 이웃을 보고 클래스를 결정할지를 정하는 하이퍼파라미터입니다.
    * **작은 k (e.g., k=1)**: 모델이 매우 유연해지며, 훈련 데이터의 노이즈에 민감해져 **과적합(Overfitting)** 되기 쉽습니다. 결정 경계가 매우 복잡해집니다.
  
    * **큰 k (e.g., k=100)**: 주변의 많은 데이터를 참고하므로 예측이 부드러워지지만, 다른 클래스의 데이터까지 이웃으로 포함하게 되어 **과소적합(Underfitting)** 될 수 있습니다. 결정 경계가 단순해집니다.
  
3.  **특성 스케일링의 중요성**: K-NN은 거리를 기반으로 하기 때문에, 특성들의 값 범위(scale)가 다르면 모델이 왜곡될 수 있습니다.
   
    예를 들어 '가입 기간(tenure)'(0~72)과 '월 요금(MonthlyCharges)'(20~120)이 있을 때, 스케일이 더 큰 월 요금이 거리 계산에 더 큰 영향을 미치게 됩니다. 
    
    따라서 K-NN을 사용할 때는 `StandardScaler`나 `MinMaxScaler`를 사용한 **특성 스케일링이 거의 필수적**입니다. (우리의 `preprocessor` 파이프라인에는 이미 `StandardScaler`가 포함되어 있습니다!)

#### 💻 코드로 알아보기

이제 K-NN 모델을 파이프라인에 적용해 보겠습니다.

In [8]:
# K-NN 모델을 포함하는 최종 파이프라인 구축 (기본 k=5)
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier()) # n_neighbors의 기본값은 5입니다.
])

# 파이프라인 학습
knn_pipeline.fit(X_train, y_train)

# 예측 및 정확도 평가
y_pred_knn = knn_pipeline.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print(f"K-NN (k=5) 모델 정확도: {accuracy_knn:.4f}")

K-NN (k=5) 모델 정확도: 0.7637


#### ✏️ 연습문제 3: 최적의 'k' 값 찾아보기

`KNeighborsClassifier`의 `n_neighbors` 값을 `3, 5, 7, 11, 15`로 변경해가면서 K-NN 모델을 학습시키고, 각 k값에 대한 정확도를 출력하여 어떤 k값이 가장 좋은 성능을 보이는지 확인해보세요.

In [9]:
# 연습문제 3 코드
k_values = [3, 5, 7, 11, 15]
for k in k_values:
    knn_pipeline_k = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', KNeighborsClassifier(n_neighbors=k))
    ])

    # 파이프라인 학습 및 평가 코드를 작성하세요.
    knn_pipeline_k.fit(X_train, y_train)
    accuracy = knn_pipeline_k.score(X_test, y_test)
    print(f"k={k} 일 때 K-NN 모델 정확도: {accuracy:.4f}")

k=3 일 때 K-NN 모델 정확도: 0.7537
k=5 일 때 K-NN 모델 정확도: 0.7637
k=7 일 때 K-NN 모델 정확도: 0.7651
k=11 일 때 K-NN 모델 정확도: 0.7786
k=15 일 때 K-NN 모델 정확도: 0.7779


#### ✏️ 연습문제 4: 이웃에 가중치 부여하기

`KNeighborsClassifier`의 `weights` 하이퍼파라미터를 `'distance'`로 설정하면, 예측 시 가까운 이웃에게 더 큰 가중치를 부여합니다. (기본값은 `'uniform'`으로 모든 이웃이 동일한 투표권을 가집니다.) `weights='distance'`로 설정한 K-NN 모델(k=11)을 학습시키고, `'uniform'`일 때와 정확도를 비교해보세요.

예시 : `KNeighborsClassifier(n_neighbors=3, weights='distance')`

In [10]:
# 연습문제 4 코드
knn_pipeline_dist = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # k=11, weights='distance'로 설정
    ('classifier', KNeighborsClassifier(n_neighbors=11, weights='distance'))
])

# 파이프라인 학습 및 평가 코드를 작성하세요.
knn_pipeline_dist.fit(X_train, y_train)
accuracy_dist = knn_pipeline_dist.score(X_test, y_test)
print(f"k=11, weights='distance' 모델 정확도: {accuracy_dist:.4f}")

k=11, weights='distance' 모델 정확도: 0.7722


---

### 4. 결정 경계(Decision Boundary) 시각화

모델이 데이터를 어떻게 구분하는지 직접 눈으로 확인하면, 모델의 특성을 훨씬 깊게 이해할 수 있습니다. 

결정 경계는 특성 공간에서 클래스를 나누는 '선' 또는 '면'을 의미합니다.

다만, 우리 데이터는 수십 개의 특성(차원)을 가지고 있어 그대로 시각화할 수 없습니다. 

따라서 **주성분 분석(PCA, Principal Component Analysis)** 이라는 차원 축소 기법을 사용하여 데이터의 주요 패턴을 유지한 채 2개의 특성(2차원)으로 압축한 뒤, 이 2차원 평면상에 결정 경계를 그려보겠습니다.

#### 💻 코드로 알아보기

결정 경계를 그리기 위한 헬퍼(helper) 함수와 PCA가 포함된 새로운 파이프라인을 정의하겠습니다.

In [11]:
X_train.shape

(5634, 19)

In [12]:
preprocessor.fit_transform(X_train).shape

(5634, 45)

In [15]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import plotly.graph_objects as go

# 1. PCA를 사용하여 데이터를 2차원으로 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(preprocessor.fit_transform(X_train))


In [16]:
X_pca.shape

(5634, 2)

In [19]:
X_pca

array([[-0.1431894 , -0.59347442],
       [-0.51281954, -0.95793988],
       [-1.02566642,  0.13208244],
       ...,
       [ 1.5743875 , -1.2282861 ],
       [-2.72467437,  1.622901  ],
       [-3.35798613,  0.60009308]])

In [18]:

# 2. 2차원 데이터로 모델 재학습 (로지스틱 회귀)
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_pca, y_train)

LogisticRegression(max_iter=1000)

In [20]:
# 3. 2차원 평면에 격자(grid) 생성
x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

In [21]:
xx.shape, yy.shape # xx, yy는 2차원 배열, 즉 격자 포인트의 x, y 좌표

((81, 96), (81, 96))

In [23]:
xx.ravel()

array([-4.48762139, -4.38762139, -4.28762139, ...,  4.81237861,
        4.91237861,  5.01237861])

In [50]:
np.c_[xx.ravel(), yy.ravel()].shape # 1차원 배열, 즉 격자 포인트의 x, y 좌표 

(7776, 2)

In [24]:
xx.ravel().shape

(7776,)

In [26]:
# 격자 포인트의 x, y 좌표를 2차원 배열로 변환
grid = np.c_[xx.ravel(), yy.ravel()]
grid.shape

(7776, 2)

**ravel() 함수란?**

`ravel()` 함수는 NumPy에서 다차원 배열을 1차원 배열로 평탄화(flatten)하는 함수입니다.

**예시:**
- `xx`는 (81, 96) 모양의 2차원 배열
- `xx.ravel()`은 (7776,) 모양의 1차원 배열로 변환 (81 × 96 = 7776)
- `np.c_[]`는 두 배열을 열 방향으로 결합하여 (7776, 2) 모양의 배열을 만듦

이렇게 변환하는 이유는 머신러닝 모델이 격자의 각 점에 대해 예측을 수행하기 위해서는 (n_samples, n_features) 형태의 2차원 배열이 필요하기 때문입니다.

In [27]:
# 4. 격자 위의 모든 점에 대해 예측 수행
Z = logistic_model.predict(grid)
Z.shape

(7776,)

In [28]:
xx.shape

(81, 96)

In [29]:
Z = Z.reshape(xx.shape) 
Z.shape

(81, 96)

**reshape() 함수를 사용하는 이유**

`Z = Z.reshape(xx.shape)`를 사용하는 이유를 설명하겠습니다.

**현재 상황:**
- `Z`는 격자의 모든 점에 대한 예측 결과로 (7776,) 모양의 1차원 배열
- `xx`, `yy`는 각각 (81, 96) 모양의 2차원 격자 배열

**reshape이 필요한 이유:**
1. **시각화를 위한 형태 변환**: Contour plot을 그리기 위해서는 Z 값이 격자와 같은 2차원 형태여야 함
2. **격자 구조 복원**: 1차원으로 평탄화된 예측 결과를 원래 격자 구조로 되돌림
3. **좌표 매핑**: 각 격자 점의 (x, y) 좌표와 해당 점의 예측값을 올바르게 매핑

**변환 과정:**
- `Z.shape`: (7776,) → (81, 96)
- 이제 `Z[i, j]`는 격자 점 `(xx[i, j], yy[i, j])`에서의 예측값을 의미

이렇게 변환해야만 Plotly의 Contour plot에서 x, y 좌표와 z 값이 올바르게 매핑되어 결정 경계를 정확히 시각화할 수 있습니다.



In [30]:
# 5. Plotly를 사용하여 결정 경계와 데이터 포인트 시각화
fig = go.Figure()

# 결정 경계 (Contour plot)
fig.add_trace(go.Contour(x=xx[0], y=yy[:, 0], z=Z,
                            colorscale=[[0, 'lightcoral'], [1, 'lightskyblue']],
                            showscale=False, opacity=0.5))

# 데이터 포인트 (Scatter plot)
scatter_df = pd.DataFrame({'PCA1': X_pca[:, 0], 'PCA2': X_pca[:, 1], 'Churn': y_train})

# 각 클래스별로 별도로 플롯 추가
for class_value in scatter_df['Churn'].unique():
    class_data = scatter_df[scatter_df['Churn'] == class_value]
    color = 'blue' if class_value == 0 else 'red'
    name = 'No Churn' if class_value == 0 else 'Churn'
    
    fig.add_trace(go.Scatter(
        x=class_data['PCA1'], 
        y=class_data['PCA2'],
        mode='markers',
        marker=dict(color=color, size=6, opacity=0.7),
        name=name,
        showlegend=True
    ))

# 레이아웃 설정
fig.update_layout(
    title='로지스틱 회귀 결정 경계 (PCA 2차원)',
    xaxis_title='첫 번째 주성분',
    yaxis_title='두 번째 주성분',
    width=800,
    height=600
)

fig.show()



이제 이 과정을 함수로 만들어보자.

In [31]:
# 결정 경계 시각화 함수
def plot_decision_boundary(model_pipeline, X, y, title):
    # 1. PCA를 사용하여 데이터를 2차원으로 축소
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(preprocessor.fit_transform(X)) # 전처리 후 PCA 적용

    # 2. 2차원 데이터로 모델 재학습
    # 파이프라인의 모델 부분만 가져와서 학습
    model = model_pipeline.named_steps['classifier']
    model.fit(X_pca, y)

    # 3. 2차원 평면에 격자(grid) 생성
    x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
    y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))

    # 4. 격자 위의 모든 점에 대해 예측 수행
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # 5. Plotly를 사용하여 결정 경계와 데이터 포인트 시각화
    fig = go.Figure()

    # 결정 경계 (Contour plot)
    fig.add_trace(go.Contour(x=xx[0], y=yy[:, 0], z=Z,
                             colorscale=[[0, 'lightcoral'], [1, 'lightskyblue']],
                             showscale=False, opacity=0.5))

    # 데이터 포인트 (Scatter plot)
    scatter_df = pd.DataFrame({'PCA1': X_pca[:, 0], 'PCA2': X_pca[:, 1], 'Churn': y})
    
    # 각 클래스별로 별도로 플롯 추가
    for class_value in scatter_df['Churn'].unique():
        class_data = scatter_df[scatter_df['Churn'] == class_value]
        color = 'blue' if class_value == 0 else 'red'
        name = 'No Churn' if class_value == 0 else 'Churn'
        
        fig.add_trace(go.Scatter(
            x=class_data['PCA1'], 
            y=class_data['PCA2'],
            mode='markers',
            marker=dict(color=color, size=6, opacity=0.7),
            name=name,
            showlegend=True
        ))

    fig.update_layout(title_text=title,
                      legend_title_text='Churn',
                      xaxis_title="첫 번째 주성분 (PCA 1)",
                      yaxis_title="두 번째 주성분 (PCA 2)", width=800, height=600)
    fig.show()

In [34]:
# 로지스틱 회귀 결정 경계 시각화
plot_decision_boundary(lr_pipeline, X_train, y_train, "로지스틱 회귀 결정 경계")

In [35]:
# K-NN (k=11) 결정 경계 시각화
knn_pipeline_11 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=11))
])
plot_decision_boundary(knn_pipeline_11, X_train, y_train, "K-NN (k=11) 결정 경계")

* **로지스틱 회귀**는 직선(선형)으로 클래스를 구분하려는 경향을 보입니다.
* **K-NN**은 데이터의 분포에 따라 더 복잡하고 구불구불한(비선형) 경계를 만듭니다.

#### ✏️ 연습문제 5: k값에 따른 결정 경계 변화 관찰하기

k값이 모델의 복잡도와 결정 경계에 어떤 영향을 미치는지 직접 확인해봅시다.

 `k=1` (가장 복잡한 모델)과 `k=101` (매우 단순한 모델)일 때의 K-NN 결정 경계를 각각 시각화하고, 두 그래프의 차이점을 설명해보세요.

 HINT: `pipeline`과 `plot_decision_boundary` 함수를 사용하세요.

In [36]:
# 연습문제 5 코드
# k=1 일 때 결정 경계 시각화
knn_pipeline_1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=1))
])
plot_decision_boundary(knn_pipeline_1, X_train, y_train, "K-NN (k=1) 결정 경계 - 과적합 예시")

# k=101 일 때 결정 경계 시각화
knn_pipeline_101 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=101))
])
plot_decision_boundary(knn_pipeline_101, X_train, y_train, "K-NN (k=101) 결정 경계 - 과소적합 예시")

# 두 그래프의 차이점을 마크다운 셀에 설명해보세요.
# 예: k=1일 때는 데이터 포인트 하나하나에 민감하게 반응하여 결정 경계가 매우 복잡하고 들쭉날쭉합니다. 이는 과적합의 특징입니다.
# 반면 k=101일 때는 경계가 매우 부드럽고 단순해져, 두 클래스를 잘 구분하지 못하는 과소적합 경향을 보입니다.